# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [40]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import cufflinks as cf
from lifelines import KaplanMeierFitter

cf.go_offline()

In [19]:
data = pd.read_csv('../data/attrition.csv')
data.columns


Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [20]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

In [21]:
rates = survival(data, 'Gender', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [22]:
rates = survival(data, 'Gender', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Time at company (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [23]:
rates = survival(data, 'Gender', 'YearsSinceLastPromotion', 'Attrition')

rates.iplot(kind='line', xTitle='Time since last promotion (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [24]:
rates = survival(data, 'MaritalStatus', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='YEars working at company (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [25]:
rates = survival(data, 'MaritalStatus', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [43]:
data['concat'] = data.Gender + " " + data.MaritalStatus

In [44]:
rates = survival(data, 'concat', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [28]:
rates = survival(data, 'JobSatisfaction', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Time at Company (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [29]:
rates = survival(data, 'Department', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Time at company (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [30]:
df = data[data["Department"]=="Sales"]

In [31]:
rates = survival(df, 'JobRole', 'YearsAtCompany', 'Attrition')

rates.iplot(kind='line', xTitle='Time at company (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [37]:
data["cut"] = pd.qcut(data.HourlyRate, 5, labels=["Very Low", "Low", "Moderate", "High", "Very High"])

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,concat,cut
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,0,8,0,1,6,4,0,5,FemaleSingle,Very High
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,1,10,3,3,10,7,1,7,MaleMarried,Moderate
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,0,7,3,3,0,0,0,0,MaleSingle,Very High
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,0,8,3,3,8,7,3,0,FemaleMarried,Low
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,1,6,3,3,2,2,2,2,MaleMarried,Very Low
5,32,0,Travel_Frequently,1005,Research & Development,2,2,Life Sciences,1,8,...,0,8,2,2,7,7,3,6,MaleSingle,High
6,59,0,Travel_Rarely,1324,Research & Development,3,3,Medical,1,10,...,3,12,3,2,1,0,0,0,FemaleMarried,High
7,30,0,Travel_Rarely,1358,Research & Development,24,1,Life Sciences,1,11,...,1,1,2,3,1,0,0,0,MaleDivorced,Moderate
8,38,0,Travel_Frequently,216,Research & Development,23,3,Life Sciences,1,12,...,0,10,2,3,9,7,1,8,MaleSingle,Very Low
9,36,0,Travel_Rarely,1299,Research & Development,27,3,Medical,1,13,...,2,17,3,2,7,7,7,7,MaleMarried,Very High


In [39]:
rates = survival(data, 'cut', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.

In [46]:
data["Overtime"] = np.where(data.OverTime == "Yes", "Overtime", "Regular Hours")
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,concat,cut,overtime,Overtime
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,0,1,6,4,0,5,Female Single,Very High,Overtime,Overtime
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,3,3,10,7,1,7,Male Married,Moderate,regular hours,Regular Hours
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,3,3,0,0,0,0,Male Single,Very High,Overtime,Overtime
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,3,8,7,3,0,Female Married,Low,Overtime,Overtime
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,3,3,2,2,2,2,Male Married,Very Low,regular hours,Regular Hours


In [54]:
data['concat2'] = data.Overtime + " " + data.BusinessTravel


In [56]:
rates = survival(data, 'concat2', 'Age', 'Attrition')

rates.iplot(kind='line', xTitle='Age (Years)', yTitle='Retention Rate',
            title='Retention Rates by Tenure and Gender')

## 